In [47]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
from langchain.globals import set_llm_cache
from langchain.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path=".demo.llm.db"))

In [49]:
ticker = "IBM"
company_name = "IBM"

In [50]:
from llm_agents_introduction.alpha_vantage import AlphaVantageService

alpha_vantage = AlphaVantageService.create()

stock_prices = alpha_vantage.fetch_daily(ticker)
stock_prices[:5]

[TimeSeriesDaily(date=datetime.date(2024, 6, 28), open=170.85, high=173.46, low=170.53, close=172.95, volume=4193459),
 TimeSeriesDaily(date=datetime.date(2024, 6, 27), open=171.12, high=172.5, low=170.48, close=170.85, volume=2894001),
 TimeSeriesDaily(date=datetime.date(2024, 6, 26), open=171.28, high=172.68, low=170.41, close=171.87, volume=2779016),
 TimeSeriesDaily(date=datetime.date(2024, 6, 25), open=175.14, high=175.7526, low=171.42, close=172.6, volume=4119267),
 TimeSeriesDaily(date=datetime.date(2024, 6, 24), open=175.0, high=178.4599, low=174.15, close=175.01, volume=4864735)]

In [51]:
import pandas as pd
from dataclasses import asdict

df = pd.DataFrame([asdict(price) for price in stock_prices])
df

,date,open,high,low,close,volume
0,2024-06-28,170.85,173.4600,170.53,172.95,4193459
1,2024-06-27,171.12,172.5000,170.48,170.85,2894001
2,2024-06-26,171.28,172.6800,170.41,171.87,2779016
3,2024-06-25,175.14,175.7526,171.42,172.60,4119267
4,2024-06-24,175.00,178.4599,174.15,175.01,4864735
...,...,...,...,...,...,...
6199,1999-11-05,92.75,92.9400,90.19,90.25,13737600
6200,1999-11-04,94.44,94.4400,90.00,91.56,16697600
6201,1999-11-03,95.87,95.9400,93.50,94.37,10369100
6202,1999-11-02,96.75,96.8100,93.69,94.81,11105400


In [52]:
from datetime import datetime, timedelta

years = 1

today = datetime.today()
start_date = (today - timedelta(days=int(365 * years))).date()

In [53]:
import plotly.express as px

last_year_prices = df[df["date"] >= start_date]
graph = px.line(
    last_year_prices,
    x="date",
    y="close",
    title=f"{company_name} stock price",
    labels={"close": "Price (USD)"},
)

graph.show()

In [54]:
from tempfile import NamedTemporaryFile

with NamedTemporaryFile(delete=False, suffix=".png") as graph_file:
    graph.write_image(graph_file.file)
    graph_file_name = graph_file.name

graph_file_name

'/var/folders/sv/dpv4tt9d45n8sv26lclcw_zr0000gn/T/tmp18shslq0.png'

In [55]:
from llm_agents_introduction.chains.stock_price_chart_reviewer import (
    create_stock_price_chart_reviewer,
    StockPriceChartReviewerInput,
)

stock_price_reviewer = create_stock_price_chart_reviewer()
stock_price_reviewer_input = StockPriceChartReviewerInput(
    from_date=start_date,
    to_date=today,
    symbol=ticker,
    image_path=graph_file_name,
)

review_items = stock_price_reviewer.invoke(stock_price_reviewer_input)

review_items

[{'start_date': datetime.date(2023, 7, 1),
  'end_date': datetime.date(2023, 11, 1),
  'notes': 'Stock price shows a general upward trend with some fluctuations.'},
 {'start_date': datetime.date(2024, 1, 1),
  'end_date': datetime.date(2024, 3, 1),
  'notes': 'Stock price experiences a significant increase.'},
 {'start_date': datetime.date(2024, 3, 1),
  'end_date': datetime.date(2024, 6, 30),
  'notes': 'Stock price shows a downward trend with some fluctuations.'}]

In [56]:
from random import randint
from copy import deepcopy


def random_color():
    return f"#{randint(0, 0xFFFFFF):06x}"


annotated_graph = deepcopy(graph)

for idx, review_item in enumerate(review_items):
    annotated_graph.add_vrect(
        x0=review_item["start_date"],
        x1=review_item["end_date"],
        fillcolor=random_color(),
        opacity=0.5,
        layer="below",
        line_width=0,
        annotation_text=idx + 1,
    )

    print(f'{idx + 1}: {review_item["notes"]}')

annotated_graph.show()

1: Stock price shows a general upward trend with some fluctuations.
2: Stock price experiences a significant increase.
3: Stock price shows a downward trend with some fluctuations.


In [57]:
from dataclasses import asdict

first_annotation = review_items[0]

related_news = alpha_vantage.fetch_relevant_news(
    ticker, first_annotation["start_date"], first_annotation["end_date"]
)

print(
    f'Have {len(related_news)} news items in the range {first_annotation["start_date"]} - {first_annotation["end_date"]}'
)
related_news[:5]

Have 397 news items in the range 2023-07-01 - 2023-11-01


[NewsLink(title='IBM Stock Inches Higher On Q3 Earnings: The Details - IBM  ( NYSE:IBM ) ', url='https://www.benzinga.com/news/earnings/23/10/35425759/ibm-stock-inches-higher-on-q3-earnings-the-details', summary="International Business Machines Corp IBM reported third-quarter financial results after the bell. Here's a look at the key highlights from the quarter. Q3 Earnings: IBM said third-quarter revenue increased 4.6% year-over-year to $14.8 billion, versus the consensus estimate of $14.728 billion, ...", overall_sentiment_score=0.280396, ticker_sentiment=[TickerSentiment(ticker='IBM', relevance_score=0.876064, ticker_sentiment_score=0.372697)]),
 NewsLink(title='IBM Considers Using In-House AI Chips to Reduce Cloud Service Costs - IBM  ( NYSE:IBM ) ', url='https://www.benzinga.com/news/23/07/33197115/ibm-considers-using-in-house-ai-chips-to-reduce-cloud-service-costs', summary='International Business Machines Corp IBM is weighing the use of artificial intelligence chips designed in-

In [58]:
from itertools import islice
from llm_agents_introduction.alpha_vantage import NewsLink
from llm_agents_introduction.article_summarization import news_links_with_text

links_with_text: list[(NewsLink, str)] = list(
    islice(news_links_with_text(related_news), 30)
)

links_with_text

Downloading: https://www.benzinga.com/news/earnings/23/10/35425759/ibm-stock-inches-higher-on-q3-earnings-the-details
Extracting: https://www.benzinga.com/news/earnings/23/10/35425759/ibm-stock-inches-higher-on-q3-earnings-the-details
Downloading: https://www.benzinga.com/news/23/07/33197115/ibm-considers-using-in-house-ai-chips-to-reduce-cloud-service-costs
Extracting: https://www.benzinga.com/news/23/07/33197115/ibm-considers-using-in-house-ai-chips-to-reduce-cloud-service-costs
Downloading: https://www.benzinga.com/markets/equities/23/08/33894634/ibm-extends-partnership-with-microsoft-to-accelerate-generative-ai-deployment
Extracting: https://www.benzinga.com/markets/equities/23/08/33894634/ibm-extends-partnership-with-microsoft-to-accelerate-generative-ai-deployment
Downloading: https://www.benzinga.com/markets/equities/23/08/34001471/ibms-portfolio-makeover-divests-weather-company-assets-to-francisco-partners
Extracting: https://www.benzinga.com/markets/equities/23/08/34001471/ibm

[(NewsLink(title='IBM Stock Inches Higher On Q3 Earnings: The Details - IBM  ( NYSE:IBM ) ', url='https://www.benzinga.com/news/earnings/23/10/35425759/ibm-stock-inches-higher-on-q3-earnings-the-details', summary="International Business Machines Corp IBM reported third-quarter financial results after the bell. Here's a look at the key highlights from the quarter. Q3 Earnings: IBM said third-quarter revenue increased 4.6% year-over-year to $14.8 billion, versus the consensus estimate of $14.728 billion, ...", overall_sentiment_score=0.280396, ticker_sentiment=[TickerSentiment(ticker='IBM', relevance_score=0.876064, ticker_sentiment_score=0.372697)]),
  'International Business Machines Corp IBM reported third-quarter financial results after the bell. Here\'s a look at the key highlights from the quarter.\nQ3 Earnings: IBM said third-quarter revenue increased 4.6% year-over-year to $14.8 billion, versus the consensus estimate of $14.728 billion, according to Benzinga Pro.\nThe company rep

In [59]:
from typing import Iterator, Optional, Sequence
from llm_agents_introduction.chains.news_article_summarizer import (
    create_news_article_summarizer,
    NewsArticleSummarizerInput,
)

news_article_summarizer = create_news_article_summarizer()


trend = first_annotation["notes"]
print(trend)


def trend_summary_from_article(
    link: NewsLink, text: str
) -> Optional[tuple[NewsLink, list[str]]]:
    summarizer_input = NewsArticleSummarizerInput(
        company_name=company_name, trend=trend, title=link.title, article_content=text
    )

    print(f"Summarizing article: {link.title}")
    summary_points = news_article_summarizer.invoke(summarizer_input)

    if summary_points:
        return link, summary_points
    else:
        return None


def summarize_links(
    links_with_text: Sequence[tuple[NewsLink, str]],
) -> Iterator[tuple[NewsLink, list[str]]]:
    for link, text in links_with_text:
        summary = trend_summary_from_article(link, text)
        if summary:
            yield summary


first_summaries = list(islice(summarize_links(links_with_text), 10))

first_summaries

Stock price shows a general upward trend with some fluctuations.
Summarizing article: IBM Stock Inches Higher On Q3 Earnings: The Details - IBM  ( NYSE:IBM ) 
Summarizing article: IBM Considers Using In-House AI Chips to Reduce Cloud Service Costs - IBM  ( NYSE:IBM ) 
Summarizing article: IBM Extends Partnership With Microsoft To Accelerate Generative AI Deployment - IBM  ( NYSE:IBM ) 
Summarizing article: IBM's Portfolio Makeover - Divests Weather Company Assets To Francisco Partners - IBM  ( NYSE:IBM ) 
Summarizing article: IBM Dumps Weather Business as Transformation Continues
Summarizing article: IBM's Double-Digit Growth In TP And Data & AI Not Sustainable, Predicts Analyst - IBM  ( NYSE:IBM ) 
Summarizing article: IBM Had A Strong Q3, But Some Analysts Have Lowered Estimates On The Stock - Here's Why - IBM  ( NYSE:IBM ) 
Summarizing article: This Dividend Stock Yielding 5% Is a Screaming Buy
Summarizing article: Could IBM Stock Soar 30%?
Summarizing article: This Dividend Stock I

[(NewsLink(title='IBM Stock Inches Higher On Q3 Earnings: The Details - IBM  ( NYSE:IBM ) ', url='https://www.benzinga.com/news/earnings/23/10/35425759/ibm-stock-inches-higher-on-q3-earnings-the-details', summary="International Business Machines Corp IBM reported third-quarter financial results after the bell. Here's a look at the key highlights from the quarter. Q3 Earnings: IBM said third-quarter revenue increased 4.6% year-over-year to $14.8 billion, versus the consensus estimate of $14.728 billion, ...", overall_sentiment_score=0.280396, ticker_sentiment=[TickerSentiment(ticker='IBM', relevance_score=0.876064, ticker_sentiment_score=0.372697)]),
  ['IBM reported third-quarter revenue increased 4.6% year-over-year to $14.8 billion, surpassing the consensus estimate.',
   'The company reported quarterly adjusted earnings of $2.20 per share, beating estimates of $2.13 per share.',
   "IBM's software and consulting revenue showed significant growth, with software revenue up 8% and cons

In [60]:
from llm_agents_introduction.chains.summaries_summarizer import (
    create_summaries_summarizer,
    SummariesSummarizerInput,
)

summaries_summarizer = create_summaries_summarizer()

final_summarize_input = SummariesSummarizerInput(
    trend=trend,
    company_name=company_name,
    title_with_points=[(link.title, summary) for link, summary in first_summaries],
)

final_summary = summaries_summarizer.invoke(final_summarize_input)

print(final_summary)

**IBM's Upward Momentum: Earnings, AI, and Strategic Shifts Drive Growth**

IBM's stock is climbing, fueled by strong earnings, strategic divestitures, and a focus on AI and cloud services. Beating revenue and earnings estimates, leveraging in-house AI chips, and extending partnerships with tech giants like Microsoft are bolstering investor confidence. High-margin software growth, cost-saving measures, and a robust dividend yield make IBM a compelling buy, despite some analyst caution.
